# Project 3
## Joshua Anderson

## General Aspects
1. There will be 10 periods. Players begin with 70 health.
2. Each period, the player harvests some amount of money (this amount is detailed below)
3. After harvesting, the player's health degenerates (this amount is detailed below)
4. After health degeneration, the player must spend money on Health Investments and Life Investments. Money spent on Health Investments increases health, while money spent on Life 5. 5. Investments gives the player Life Enjoyment. Any money not spent carries over into the next period.
6. A player dies if their health ever goes below 0. If a player dies, they receive 0 Life Enjoyment for the remaining periods.
7. The goal is to maximize total Life Enjoyment across all periods.

## Functions
 

Harvesting: 
1. The player earns income by harvesting black dots in a region designated by M x N cells. In our parameters (M = N = 100)
2. The player can select any set of contiguous W columns to harvest. In our parameters (W = 10)
3. When fully healthy, there are T black dots dispersed across M x W cells each period. Each black dot is worth v. In our parameters T = 100 and v = 1
4. The number of rows that can be harvested each period are given by: HarvestRows(H) = M $(1 - \gamma \frac{100 - H}{100})$ where $(\gamma = 1)$
5. The number of rows are reduced by disabling the rows in the upper and lower regions of the selected columns.
6. With these parameters, if health is 50 at the start of the period, the player has only 500 cells in which they can harvest black dots. With 80 health, they have 800 cells.
 

Degeneration:

Each period, the player loses (10 + CurrentPeriod) in health. i.e. 11 health the first period, 12 the second period, up to 20 in the last period.

Health Regeneration:

The equation for the amount of health regained given a certain Health Investment, I, and health after harvesting, H, is given by: 
$$ HealthRegained(I, H) = 100 \left( \frac{e^{k \cdot I}}{e^{k \cdot I} + \frac{100-H}{H}}\right)  - H $$

where (k = 0.01021).  Health cannot exceed 100, and is always rounded down to the nearest integer. 

Life Enjoyment:

The equation for the amount of Life Enjoyment given a certain Life Investment, L, is given by: 

$$ LifeEnjoyment(L, CurrentHealth) = c \left( \frac{CurrentHealth}{100} \right) \left( \frac{L}{L + \alpha} \right) $$

where (c = 464.53) and ($\alpha$ = 32).

CurrentHealth is the health the player has during the investment phase **INCLUDING** the amount regained this period through investments in health.

In [27]:
import numpy as np
import pandas as pd
import math
import random

np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

# Constants
periods = 10
population_size = 50

starting_health = 70
min_health = 0

starting_enjoyment = 0

mutate_prob = 0.05
num_children = int(population_size * .75)


In [22]:
### player class ###
class Player:
    def __init__(self, health, enjoyment, money, I_props, L_props, money_props):
        self.health = health
        self.enjoyment = enjoyment
        self.money = money
        self.I_props = I_props
        self.L_props = L_props
        self.money_props = money_props

In [20]:
### simulation functions ###

# Harvesting
def harvest(health, M = 100, N = 100, W = 10, T = 100, v = 1, gamma = 1):
    dots_per_cell = T / (M * W)

    num_player_rows = M * (1 - (gamma * ((100 - health) / 100)))
    return num_player_rows * W * dots_per_cell * v
        
# Degeneragtion
def degenerate(curr_period, P = 10):
    return P + curr_period
    

# Health Regeneration
def regenerate(I, health, k = 0.01021, max_health = 100):
        e_kI = math.exp(k*I)
        H_frac = (max_health - health) / health
        
        health_regen = math.floor(max_health * (e_kI / (e_kI + H_frac)) - health)

        if health_regen > 0:
            if health_regen > max_health:
                return max_health
            else:
                return health_regen
        
        return 0

# Life Enjoyment - Fitness Function
def enjoyment(L, health, c = 464.53, alpha = 32):
        return c * (health / 100) * (L / (L + alpha))

# make investments
def invest_pop(players):
    for player in players:
        if player.health > min_health:
            I, L = player.invest()
            player.health += regenerate(I, player.health)
            player.enjoyment += enjoyment(L, player.health)

# helper function to print population
def print_players(players):
    print("    H  -   LE   -   I  -   L")
    print(np.array([(player.health, player.enjoyment, player.I_prop, player.L_prop) for player in players]))

In [85]:
# generate population
def generate_players(starting_health, population_size, num_periods):
    players = []
    for _ in range(population_size):
        health_list = [starting_health]
        enjoyment_list = [0]
        money_list = [harvest(starting_health)]
        L_prop_list = [random.random()]
        I_prop_list = [random.uniform(0, L_prop_list[0])]
        money_prop_list = [1 - (I_prop_list[0] + L_prop_list[0])]

        # simulate all periods for player
        # ignore first period
        for i in range(num_periods-1):

            if health_list[i] > min_health:
                L_prop = round(random.random(),2)
                int(100*L_prop)
                I_prop = random.randint(0, int(100*L_prop)) / 100
                money_prop = 1 - (L_prop + I_prop)

                enjoyment_list.append(enjoyment_list[i] + enjoyment(L_prop * money_list[i], health_list[i]))
                money_list.append((money_list[i] * money_prop) + harvest(i + 1))

                new_health = health_list[i] - degenerate(i+1) + regenerate(I_prop, health_list[i])
                if new_health > min_health:
                    health_list.append(new_health)
                else:
                    health_list.append(0)

                L_prop_list.append(L_prop)
                I_prop_list.append(I_prop)
                money_prop_list.append(money_prop)
            
            else:
                enjoyment_list.append(enjoyment_list[i])
                health_list.append(health_list[i])
                money_list.append(money_list[i])

                L_prop_list.append(0)
                I_prop_list.append(0)
                money_prop_list.append(0)
            
        players.append(Player(health_list, enjoyment_list, money_list, I_prop_list, L_prop_list, money_prop_list))
    return players

# helper function to print players
def print_players(players):
    data = [(player.health, player.enjoyment, player.money, player.I_props, player.L_props, player.money_props) for player in players]
    print(np.array(data))

In [84]:
players = generate_players(starting_health, 2, periods)

print_players(players)
# curr_pop = players
# for curr_period in range(10):
#     curr_pop = ga_iteration(curr_pop, curr_period, mutate_prob, population_size)

# print_players(curr_pop)

[[[   70.      59.      47.      34.      20.       5.       0.
       0.       0.       0.  ]
  [    0.     102.36   266.38 -2321.17 -2262.64 -2263.02 -2260.01
   -2260.01 -2260.01 -2260.01]
  [   70.      54.2    -34.31    27.71    -0.16     5.06     5.85
       5.85     5.85     5.85]
  [    0.34     0.03     0.79     0.86     0.47     0.57     0.09
       0.       0.       0.  ]
  [    0.69     0.21     0.88     0.86     0.68     0.83     0.94
       0.       0.       0.  ]
  [   -0.03     0.76    -0.67    -0.72    -0.15    -0.4     -0.03
       0.       0.       0.  ]]

 [[   70.      59.      47.      34.      20.       5.       0.
       0.       0.       0.  ]
  [    0.      72.     244.27   235.18   246.25   246.35   250.9
     250.9    250.9    250.9 ]
  [   70.      57.      -1.42     3.01     3.37     8.3      0.27
       0.27     0.27     0.27]
  [    0.01     0.07     0.11     0.11     0.41     0.01     0.75
       0.       0.       0.  ]
  [    0.07     0.13     0.95    